In [1]:
from pyqubo import Binary, Constraint, SubH
from pyqubo import UnaryEncInteger,LogEncInteger
import numpy as np
import csv
import time

In [2]:
def csv_to_matrix(filename, size):
    num_rows, num_cols = size
    matrix = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            
            if len(row) != num_cols:
                raise ValueError("Number of columns in row does not match expected number of columns")
            matrix_row = []
            for value in row:
                try:
                    matrix_row.append(float(value))
                except ValueError:
                    matrix_row.append(value)
            matrix.append(matrix_row)
    if len(matrix) != num_rows:
        raise ValueError("Number of rows in matrix does not match expected number of rows")
    return matrix

## We input the weight coefficient calculated by DFT calculations from csv file

In [3]:
# In our proof-of concept work, there are 5 sites and 11 different molecular structure to choose.
num_of_position, num_of_mol = 5,11

# The contribution from one functional group.
w = csv_to_matrix('0117data_one_functional group.csv', (5,11))

# The contribution from the interactions of two functional groups
W10 = csv_to_matrix('0117data_R1R2_double_functional group.csv', (11,11))
W34 = csv_to_matrix('0117data_R1R2_double_functional group.csv', (11,11))


W21 = csv_to_matrix('0117data_R2R3_double_functional group.csv', (11,11))
W23 = csv_to_matrix('0117data_R2R3_double_functional group.csv', (11,11))
# second nearest neighbor interaction
W04 = csv_to_matrix('0117data_R1R5_double_functional group.csv', (11,11))


W20 = csv_to_matrix('0117data_R1R3_double_functional group.csv', (11,11))
W24 = csv_to_matrix('0117data_R1R3_double_functional group.csv', (11,11))


W31 = csv_to_matrix('0117data_R2R4_double_functional group.csv', (11,11))


W30 = csv_to_matrix('0117data_R1R4_double_functional group.csv', (11,11))
W14 = csv_to_matrix('0117data_R1R4_double_functional group.csv', (11,11))

### Here we label variables as $x_{ij}$, where $i$ labels different sites and $j$ labels different functional groups.

In [4]:
# create the binary variables for the objective function
def create_x_vars(num_of_position, num_of_mol):
    a = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    for i in range(num_of_position):
        for j in range(num_of_mol):
            vars_name = 'x'+str(i)+'_'+str(j)
            a[i][j] = Binary(vars_name)
    return a

### We construct the objective function according to our weight coefficients.

In [5]:
# construct objective function
x = create_x_vars(num_of_position, num_of_mol)

# The variable "summ" corresponds to the summation over the total contribution from the single functional group.
summ = 0
for i in range(num_of_position):
    for j in range(num_of_mol):
        summ += w[i][j]*x[i][j]
        

# The following variables correspond to the contributions from the interaction from each pair of two functional groups.         
summ_10 = 0
summ_21 = 0
summ_23 = 0
summ_34 = 0
summ_04 = 0

summ_20 = 0
summ_24 = 0

summ_31 = 0
summ_30 = 0
summ_14 = 0


for i in range(1,num_of_mol):
    for j in range(1,num_of_mol):
        # nearest neighbor interaction
        summ_10 += W10[j][i]*x[1][j]*x[0][i]
        summ_34 += W34[j][i]*x[3][j]*x[4][i] 

        summ_21 += W21[j][i]*x[2][j]*x[1][i]  
        summ_23 += W23[j][i]*x[2][j]*x[3][i]

        summ_04 += W04[j][i]*x[0][j]*x[4][i] 

        # second nearest neighbor interaction
        summ_20 += W20[j][i]*x[2][j]*x[0][i] 
        summ_24 += W24[j][i]*x[2][j]*x[4][i]

        summ_31 += W31[j][i]*x[3][j]*x[1][i] 
        summ_30 += W30[j][i]*x[3][j]*x[0][i] 
        summ_14 += W14[j][i]*x[1][j]*x[4][i]

obj =  (summ_10 + summ_34 + summ_21 + summ_23 + summ_04 + summ_20 + summ_24 + summ_31 + summ_30 + summ_14 ) + summ  + 87.5

## We impose the constraints mentioned in the paper onto our objective function.

In [6]:
# This is the contraint for "one site can only connect to one functional group".
summ_c_all = 0
for i in range(num_of_position):
    summ_c = 0
    for j in range(num_of_mol):
        summ_c += x[i][j]
    summ_c_all += (summ_c -1)**2
    
con_1 = summ_c_all

# This is the contraint for "at most three sites could connect to non-hydrogen functional groups".
summ_h = 0
for i in range(num_of_position):
    summ_h += x[i][0]

con_2 = (summ_h -2)**2


# The total QUBO model is the combination of objective function and those two contraints.
H = SubH(obj, 'sub_h') + 70*(con_1 +con_2)
qubo, offset = H.compile().to_qubo()

ineq = [{"coefficient":1,"polynomials":[i]} for i in range(len(H.compile().variables))] + [{"coefficient":-3,"polynomials":[]}]
constraint = {"terms": ineq, "lambda":1}
one_hot_constraint = {"numbers":[10, 10, 10, 10, 10]}
penalty_polynomial = [{"coefficient":-3}] + [{"coefficient":1,"polynomials":[i]} for i in range(len(H.compile().variables))]

In [7]:
constrain_1_2, offset_1_2 = (con_1+con_2).compile().to_qubo()

## We use DA solvers to find the low energy solutions of the QUBO model.

In [8]:
from pyqubo import Binary
from pprint import pprint
import requests
import json
import re
from __future__ import print_function
from time import sleep
import copy
import numpy as np

In [9]:
# The input header for the json file
# ---------------------------
access_key = ''
post_url =   "https://api.aispf.global.fujitsu.com/da"
post_headers = {'X-Api-Key' : access_key, \
                   'Accept': 'application/json', \
                   'content-Type': 'application/json'}

rest_url = 'https://api.aispf.global.fujitsu.com'
version = 'v3'
proxies = {}

In [11]:
def req(ddic):
    response = requests.post(post_url,
            json.dumps(ddic), \
            headers=post_headers)
    print(response.json())
    print(type(response.json()))
    return response

def qubodic_to_fdic(qdic, offset):
    # pyqubo x0, x1 to Fujitsu DA BinaryPolynomial
    fdic = {}
    gdic = {}
    alist = []
    for k, v in qdic.items():
        edic = {}
        a0 = re.sub(r'^[a-zA]', '', k[0])
        
        a1 = re.sub(r'^[a-zA]', '', k[1])
        
        edic["coefficient"] = v
        if a0[0] == '[':
            a0 = a0[1]
        if a1[0] == '[':
            a1 = a1[1]
        edic["polynomials"] = [int(a0), int(a1)]
        alist.append(edic)
    edic = {}
    edic["coefficient"] = offset
    alist.append(edic)
    gdic["terms"] = alist
    fdic["binary_polynomial"] = gdic 

    return fdic


def qubodic_to_fdic_trans(qdic, offset):
    # pyqubo x0, x1 to Fujitsu DA BinaryPolynomial
    fdic = {}
    gdic = {}
    alist = []
    for k, v in qdic.items():
        edic = {}
        edic["coefficient"] = v
        edic["polynomials"] = [int(k[0]), int(k[1])]
        alist.append(edic)
    edic = {}
    edic["coefficient"] = offset
    alist.append(edic)
    gdic["terms"] = alist
    fdic["binary_polynomial"] = gdic 
    return fdic


class DA3Solver(object):
    """Digital Annealer Solver class
       Arguments:
           time_limit_sec:      .
           target_energy:       .
           num_output_solution: .
       Attributes:
           rest_url:            Digital Annealer Web API address and port 'http://<address>:<port>'.
    """

    def __init__(self, time_limit_sec=1, target_energy=64, num_output_solution=12):
        self.rest_url = rest_url
        self.access_key = access_key
        self.version = version
        self.type_num = None
        self.proxies =proxies
        self.rest_headers = {'content-type': 'application/json'}
        self.params = {}
        self.params['time_limit_sec'] = time_limit_sec
        self.params['target_energy'] = target_energy
        self.params['num_output_solution'] = num_output_solution
        self.total_elapsed_time = 0

    
    def output_request(self, qubo):
        """Find the minimum value of a Quadratic Polynomial 'poly' and return a object of SolverResponse class"""
        request = {"fujitsuDA3": self.params}
        request.update(qubo)
        headers = self.rest_headers
        headers['X-Api-Key'] = self.access_key
        return request
    
    def minimize(self, qubo):
        """Find the minimum value of a Quadratic Polynomial 'poly' and return a object of SolverResponse class"""
        request = {"fujitsuDA3": self.params}
        request.update(qubo)
        headers = self.rest_headers
        headers['X-Api-Key'] = self.access_key
        sleep(10)
        post_status = requests.post(self.rest_url + '/da/' + self.version + '/async/qubo/solve', json.dumps(request), headers=headers, proxies=self.proxies)
        jobid = post_status.json()['job_id']
        sleep(request['fujitsuDA3']['time_limit_sec'])
        response = requests.get(self.rest_url + '/da/' + self.version + '/async/jobs/result/' + jobid, headers=headers, proxies=self.proxies)
        delete_status = requests.delete(self.rest_url + '/da/' + self.version + '/async/jobs/result/' + jobid, headers=headers, proxies=self.proxies)
        if post_status.ok:
            j = response.json()
            try:
                if j[u'qubo_solution'].get(u'timing'):
                    self.total_elapsed_time = j[u'qubo_solution'][u'timing'][u'total_elapsed_time']
                if j[u'qubo_solution'][u'result_status']:
                    return SolverResponse(response)
                raise RuntimeError('result_status is false.')
            except KeyError:
                return jobid
        else:
            raise RuntimeError(response.text)
    
class SolverResponse(object):
    """Solver Response class
       Attributes:
           response:          The raw data which is a response of requests.
           answer_mode:       The distribution of solutions. When 'HISTOGRAM' is set, get_solution_list() returns a histogram of solutions.
    """
    class AttributeSolution(object):
        def __init__(self, obj):
            self.obj = obj

        def __getattr__(self, key):
            if key in self.obj:
                return self.obj.get(key)
            else:
                raise AttributeError(key)

        def keys(self):
            return self.obj.keys()
        def mini(self):
            return self._solutions

    def __init__(self, response):
        solutions = response.json()[u'qubo_solution'][u'solutions']
        self.answer_mode = 'RAW'
        self.response = response
        self._solutions = [self.AttributeSolution(d) for d in solutions]
        self._solution_histogram = []
        lowest_energy = None
        for sol in solutions:
            if lowest_energy is None or lowest_energy > sol.get(u'energy'):
                lowest_energy = sol.get(u'energy')
                self.minimum_solution = self.AttributeSolution(sol)
        for i, d in enumerate(solutions):
            if i == solutions.index(d):
                self._solution_histogram.append(copy.deepcopy(d))
            else:
                for s in self._solution_histogram:
                    if s[u'configuration'] == d[u'configuration']:
                        s[u'frequency'] += 1
                        break
        self._solution_histogram = sorted([self.AttributeSolution(d) for d in self._solution_histogram], key=lambda x: x.energy)

    def get_minimum_energy_solution(self):
        """Get a minimum energy solution"""
        return self.minimum_solution

    def get_solution_list(self):
        """Get all solution"""
        if self.answer_mode == 'HISTOGRAM':
            return self._solution_histogram
        else:
            return self._solutions

def delete_job(job_id):
    headers = {
         'X-Api-Key': str(access_key),
         'Accept': 'application/json',
        'content-type': 'application/json'
    }
    url = 'https://api.aispf.global.fujitsu.com/da/v3/async/jobs/result/'+str(job_id)
    response1 = requests.delete(url, headers=headers)

def retrive_job(job_id_str):
    headers = {
         'X-Api-Key': str(access_key),
         'Accept': 'application/json',
        'content-type': 'application/json'
    }
    url = 'https://api.aispf.global.fujitsu.com/da/v3/async/jobs/result/'+str(job_id_str)
    response1 = requests.get(url, headers=headers)
    return response1

def output_candidate_energy(result):
    # result is in the type of '__main__.SolverResponse'
    for j in range(len(result.get_solution_list())):
        key_list = list(result.get_solution_list()[j].obj['configuration'].keys())
        aa = {label_trans_invers[i]: int(result.get_solution_list()[j].obj['configuration'][i])  for i in key_list}
        print('Energy=',result.get_solution_list()[j].obj['energy'])
        print('Frequency=',result.get_solution_list()[j].obj['frequency'])
        for key, value in aa.items():
            if value ==1:
                print(key)
        print()
        
from ast import literal_eval
def get_job_on_that_day(year,month,day, response):
    date =str(year)+'-'+str(month)+'-'+str(day)
    my_dict = literal_eval(response.content.decode('utf-8'))
    for i in my_dict['job_status_list']:
        if i['start_time'][0:10]==str(date):
            print(i) 
    print(len(my_dict['job_status_list']))
def retiv_job_output(job_id):
    retriv = retrive_job(job_id)
    for i in retriv.json()['qubo_solution']['solutions']:
        answer_dic = i['configuration']
        optimal_energy = i['energy']
        time = retriv.json()['qubo_solution']['timing']
        key_list = answer_dic.keys()
        aa = {label_trans_invers[i]: int(answer_dic[i])  for i in key_list}
        for key, value in aa.items():
            if value == 1:
                print(key)
        print(optimal_energy)
    print(time)

In [12]:
# We transform the QUBO model into json format.
label_trans = {H.compile().variables[i]:str(i)   for i in range(len(H.compile().variables))}
label_trans_invers = {str(i): H.compile().variables[i] for i in range(len(H.compile().variables)) }
qubo_trans = {(label_trans[list(qubo)[i][0]] , label_trans[list(qubo)[i][1]]) : qubo[(list(qubo)[i][0] , list(qubo)[i][1])] for i in range(len(list(qubo)))}
fdic_trans = qubodic_to_fdic_trans(qubo_trans, offset)

In [17]:
# We upload the json file to the DA solver.
solver = DA3Solver()
solver.access_key = access_key
solver.rest_url = rest_url
solver.version = version
solver.proxies = proxies
results = solver.minimize(fdic_trans)

In [18]:
print(results)
retriv = retrive_job(results)
retrive_job(results).json()

0cb62fc0-cefb-4819-a36d-2c921a3d5b6a-233255911485778


{'status': 'Running'}

In [19]:
# We transform the output sample from the DA solver.
samples = retrive_job(results).json()['qubo_solution']['solutions']
for i in samples:
    a = i['configuration']
    output = [ int(a[str(j)]) for j in range(50)]
    candidates = [label_trans_invers[str(i)] for i in range(len(output)) if output[i] !=0 ]
    print(candidates,'Energy:',i['energy'],',Frequency:',i['frequency'])

['x4_8', 'x2_2', 'x0_1'] Energy: 65.6366319899999 ,Frequency: 1
['x4_1', 'x2_2', 'x0_8'] Energy: 65.63663199 ,Frequency: 1
['x4_8', 'x2_2', 'x0_8'] Energy: 66.6035225969999 ,Frequency: 1
['x4_8', 'x1_2', 'x0_1'] Energy: 67.07523233400003 ,Frequency: 1
['x4_1', 'x3_2', 'x0_8'] Energy: 67.07523233400002 ,Frequency: 1
['x4_2', 'x2_2', 'x0_5'] Energy: 67.68582976599997 ,Frequency: 1
['x4_5', 'x2_2', 'x0_2'] Energy: 67.7128123949999 ,Frequency: 1
['x4_2', 'x3_2', 'x0_8'] Energy: 67.78501832399998 ,Frequency: 1
['x4_8', 'x1_2', 'x0_2'] Energy: 67.78501832399999 ,Frequency: 1
['x4_2', 'x1_7', 'x0_2'] Energy: 67.861062463 ,Frequency: 1
['x4_2', 'x3_7', 'x0_2'] Energy: 67.86106246299998 ,Frequency: 1
['x4_1', 'x2_2', 'x0_5'] Energy: 68.00420080899997 ,Frequency: 1


In [21]:
# The time usage for the DA solver to sample out the states.
time = retrive_job(results).json()['qubo_solution']['timing']
print(time)

{'solve_time': '2170', 'total_elapsed_time': '2186'}
